In [1]:
import keras
from keras.datasets import mnist
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [2]:
(x_train_origin, y_train_origin),(x_test_origin, y_test_origin) = mnist.load_data()
x_train, x_test = x_train_origin / 255.0, x_test_origin / 255.0

In [3]:
# if digit == 1, then label = 1, else label = 0
func = lambda x: 1 if x == 1 else 0
#y_train = list(map(func, y_train_origin))
#y_test = list(map(func, y_test_origin))

In [4]:
x_train = x_train_origin.reshape((60000, 28 * 28))
x_train = x_train_origin.astype('float32') / 255

x_test = x_test_origin.reshape((10000, 28 * 28))
x_test = x_test_origin.astype('float32') / 255

In [5]:
y_train = np.array([func(y) for y in y_train_origin])
y_test = np.array([func(y) for y in y_test_origin])

In [6]:
x_train = x_train.reshape(x_train.shape[0], -1).T
x_test = x_test.reshape(x_test.shape[0], -1).T

y_train = y_train.reshape(1,y_train.shape[0])
y_test = y_test.reshape(1,y_test.shape[0])

In [7]:
def sigmoid(z):
    
    s = 1.0/(1.0 + np.exp(-z))
    
    return s

In [8]:
def initialize_params_with_zeros(dim):
    
    w = np.zeros((dim,1))
    b = 0
    
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [9]:
def propagate(w, b, X, Y):
    
    m = X.shape[1]

    A = sigmoid(np.dot(w.T, X) + b)                                       
    loss = (1/m) * np.sum(np.square(Y - A))
    
    dL = 2*(A - Y)
    dA = A * (1 - A) * dL
    
   
    dw = (1/m) * np.dot(X, dA.T)
    db = (1/m) * np.sum(dA)

    #cost = np.squeeze(cost)
    #assert(cost.shape == ())
    grads = {"dw": dw,
             "db": db}
    return grads, loss

In [10]:
def optimize(w, b, X, Y, num_iters, batch_size, learning_rate, print_cost=False):
   
    costs = []
    m = X.shape[0]
    for i in range(num_iters):      
        for j in range(0, m, batch_size):
            X = x_train.T[j:j+batch_size]
            Y = y_train.T[j:j+batch_size]
            grads, cost = propagate(w, b, X.T, Y.T)
            dw = grads["dw"]
            db = grads["db"]
            w = w - learning_rate *  dw    
            b = b - learning_rate *  db
            costs.append(cost)
        print ("Cost after iteration %i: %f" % (i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs


In [11]:
# GRADED FUNCTION: predict

def predict(w, b, X):
   
    m = X.shape[1]
    Y_pred = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    

    A = sigmoid(np.dot(w.T, X) + b)

    for i in range(A.shape[1]):
        if A[0][i] <= 0.5:
            Y_pred[0, i] = 0
        else:
            Y_pred[0, i] = 1
                
    assert(Y_pred.shape == (1, m))
    
    return Y_pred

In [12]:
# GRADED FUNCTION: model
def model(x_train, y_train, x_test, y_test, num_iters=20, batch_size = 5, learning_rate=0.5, print_cost=False):
    
    w, b = initialize_params_with_zeros(x_train.shape[0])                          

    parameters, grads, costs = optimize(w, b, x_train, y_train, num_iters, batch_size, learning_rate, print_cost=True)     

    w = parameters["w"]
    b = parameters["b"]
    

    Y_pred_test = predict(w, b, x_test)
    Y_pred_train = predict(w, b, x_train)

    print("")

    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_pred_train - y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_pred_test - y_test)) * 100))
    repr('train accuracy')


    
    d = {"costs": costs,
         "Y_pred_test": Y_pred_test, 
         "Y_pred_train" : Y_pred_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iters": num_iters}
    
    return d

In [13]:
d = model(x_train, y_train, x_test, y_test, num_iters=10, batch_size=1, learning_rate=0.5, print_cost=True)

Cost after iteration 0: 0.000047
Cost after iteration 1: 0.005313
Cost after iteration 2: 0.000027
Cost after iteration 3: 0.001174
Cost after iteration 4: 0.002376
Cost after iteration 5: 0.001004
Cost after iteration 6: 0.000969
Cost after iteration 7: 0.000148
Cost after iteration 8: 0.000071
Cost after iteration 9: 0.000055

train accuracy: 98.72 %
test accuracy: 98.96 %
